## Q2

In [117]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize,LabelBinarizer
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import defaultdict

import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision.transforms as transforms
import torchvision
import PIL
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,random_split,Dataset
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix,accuracy_score,multilabel_confusion_matrix

In [ ]:
%pip install wandb -q
import wandb
wandb.login()

In [18]:

# Initialize WandB 
wandb.init(name='test_run', 
           project='ML_Assignment2',
           notes='This is a test run', 
           tags=['Fashion MNIST', 'Test Run'],
           entity='helloadi19138')

In [10]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)
vgg16


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
import torch
import torch.nn as nn
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 10)]) # Add custom layer with 10 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
vgg16


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import torch.optim as optim
# Define Optimizer and Loss Function
loss = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# wandb.config={
#     "learning_rate": 0.001,
#     "momentum": 0.9,
#     "batch_size": 32,
#     "epochs":20
# }

In [13]:

train_valid = pd.read_csv('/content/drive/MyDrive/fashion-mnist_train.csv').iloc[:1000,:]
lb=LabelBinarizer()
lb.fit(train_valid['label'])

test = pd.read_csv('/content/drive/MyDrive/fashion-mnist_test.csv').iloc[:100,:]
train, valid = train_test_split(train_valid, test_size=0.2,shuffle=True)

train_x, train_y = np.array(train.iloc[:,1:]).astype(np.uint8).reshape(train.shape[0],28,28), lb.transform(train['label'].values).astype(np.float64)
valid_x, valid_y = np.array(valid.iloc[:,1:]).astype(np.uint8).reshape(valid.shape[0],28,28), lb.transform(valid['label'].values).astype(np.float64)
test_x, test_y = np.array(test.iloc[:,1:]).astype(np.uint8).reshape(test.shape[0],28,28), lb.transform(test['label'].values).astype(np.float64)




print('Train set', train_x.shape, train_y.shape)
print('Valid set', valid_x.shape, valid_y.shape)
print('Test set',test_x.shape, test_y.shape)

Train set (800, 28, 28) (800, 10)
Valid set (200, 28, 28) (200, 10)
Test set (100, 28, 28) (100, 10)


In [30]:

train_transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
    ])
test_transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
    ])

class FashionMNIST(Data.Dataset):  

  def __init__(self, X, y,transform=None):
    'Initialization'
    self.X = X
    self.y = y
    self.transform=transform

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.X)

  def __getitem__(self, index):
    'Generates one sample of data'
    # Select sample
    image,label = self.X[index],self.y[index]
    X = self.transform(image)
    X=X.repeat(3,1,1)
    X=self.norm(X)    
    return X,label

  norm=transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])

train_loader = Data.DataLoader(FashionMNIST(train_x,train_y,train_transform), batch_size = 32, shuffle = True)
valid_loader = Data.DataLoader(FashionMNIST(valid_x,valid_y,test_transform), batch_size = 32, shuffle = True)
test_loader = Data.DataLoader(FashionMNIST(test_x,test_y,test_transform), shuffle = True)

In [79]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 3, 224, 224])
Labels batch shape: torch.Size([32, 10])


In [114]:
import time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
  model.to(device)
  start = time.time()
  history = []
  best_acc = 0.0

  for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
    # Set to training mode
    model.train(True)

    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0        
    val_loss = 0.0
    val_acc = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
      # print("i=",i)
      inputs = inputs.to(device)
      labels = labels.to(device)

      
      # Clean existing gradients
      optimizer.zero_grad()

      # Forward pass - compute outputs on input data using the model
      outputs = model(inputs)
      # print("forward prop out shape",outputs.shape)
      
      
      # Compute loss
      loss = loss_criterion(outputs, labels)
      # print("Computed loss= ",loss)
      
      # Backpropagate the gradients
      loss.backward()
      # print("Backprop")
      
      # Update the parameters
      optimizer.step()
      # print("Updation")

      train_loss += loss.item() * inputs.size(0)
      # Compute the accuracy
      ret, predictions = torch.max(outputs.data, 1)  #predictions= indices of max prob value in every row
      # print(predictions)
      _,label_max_idx = torch.max(labels.data,1)
      # print(label_max_idx)
      correct_counts = predictions.eq(label_max_idx)
      
      # Convert correct_counts to float and then compute the mean
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
      
      # Compute total accuracy in the whole batch and add to train_acc
      train_acc += acc.item() * inputs.size(0)
      
      # print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

    # Validation - No gradient tracking needed
    with torch.no_grad():
      # Set to evaluation mode
      model.eval()

      # Validation loop
      for j, (inputs, labels) in enumerate(valid_loader):
          # print("j=",j)
          # if use_gpu:
          #   inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
          # else:
          #   inputs, labels = Variable(inputs), Variable(labels)
          inputs = inputs.to(device)
          labels = labels.to(device)

          # Forward pass - compute outputs on input data using the model
          outputs = model(inputs)

          # Compute loss
          loss = loss_criterion(outputs, labels)

          # Compute the total loss for the batch and add it to valid_loss
          val_loss += loss.item() * inputs.size(0)

          # Calculate validation accuracy
          ret, predictions = torch.max(outputs.data, 1)
          _,label_max_idx = torch.max(labels.data,1)
          # print(label_max_idx)
          correct_counts = predictions.eq(label_max_idx)

          # Convert correct_counts to float and then compute the mean
          acc = torch.mean(correct_counts.type(torch.FloatTensor))

          # Compute total accuracy in the whole batch and add to valid_acc
          val_acc += acc.item() * inputs.size(0)

          # print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
      
    # Find average training loss and training accuracy
    avg_train_loss = train_loss/train_x.shape[0] 
    avg_train_acc = train_acc/train_x.shape[0] 

    # Find average training loss and training accuracy
    avg_valid_loss = val_loss/valid_x.shape[0]
    avg_valid_acc = val_acc/valid_x.shape[0]

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
            
    epoch_end = time.time()
    # Log the loss and accuracy values at the end of each epoch
    # wandb.log({
    #     "Epoch": epoch,
    #     "Train Loss": avg_train_loss,
    #     "Train Acc": avg_train_acc,
    #     "Valid Loss": avg_valid_loss,
    #     "Valid Acc": avg_valid_acc})

    print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
    
    # Save if the model has best accuracy till now
    #torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
              
  return model

def eval_model(model, criterion):
  model.to(device)
  test_acc=0
  pred=[]
  true=[]

  for j, (inputs, labels) in enumerate(test_loader):
    # print("j=",j)
    inputs=inputs.to(device)
    labels=labels.to(device)

    out = model(inputs)
    # print(out.shape)
    _, predictions = torch.max(out.data, 1)
    pred.append(predictions.item())
    _,label_max_idx = torch.max(labels.data,1)
    true.append(label_max_idx.item())
    correct_counts = predictions.eq(label_max_idx)
    acc = torch.mean(correct_counts.type(torch.FloatTensor))
    test_acc += acc.item() 


  return true,pred,test_acc/test_x.shape[0]


        

In [82]:
device=["cuda"]
num_epochs = 20
vgg16 = train_and_validate(vgg16, loss, optimizer, num_epochs)
torch.save(vgg16.state_dict(), 'VGG16.pt')

Epoch: 1/20
Epoch : 001, Training: Loss: 0.8590, Accuracy: 69.8750%, 
		Validation : Loss : 0.9377, Accuracy: 67.5000%, Time: 29.2505s
Epoch: 2/20
Epoch : 002, Training: Loss: 0.8336, Accuracy: 71.2500%, 
		Validation : Loss : 0.8756, Accuracy: 70.0000%, Time: 29.1370s
Epoch: 3/20
Epoch : 003, Training: Loss: 0.8226, Accuracy: 72.6250%, 
		Validation : Loss : 0.8207, Accuracy: 71.5000%, Time: 29.2138s
Epoch: 4/20
Epoch : 004, Training: Loss: 0.7902, Accuracy: 71.8750%, 
		Validation : Loss : 0.8673, Accuracy: 70.5000%, Time: 29.1190s
Epoch: 5/20
Epoch : 005, Training: Loss: 0.7766, Accuracy: 73.2500%, 
		Validation : Loss : 0.9351, Accuracy: 69.5000%, Time: 29.1531s
Epoch: 6/20
Epoch : 006, Training: Loss: 0.8158, Accuracy: 71.2500%, 
		Validation : Loss : 0.8714, Accuracy: 68.0000%, Time: 29.1577s
Epoch: 7/20
Epoch : 007, Training: Loss: 0.7366, Accuracy: 74.2500%, 
		Validation : Loss : 0.8639, Accuracy: 68.5000%, Time: 29.1553s
Epoch: 8/20
Epoch : 008, Training: Loss: 0.7392, Accura

Q2 b)

In [116]:
y_true,y_pred,test_acc= eval_model(vgg16,loss)
print("Test accuracy for VGG16 model=",test_acc)
print("Confustion matrix --> ")
print(confusion_matrix(y_true,y_pred))

Test accuracy for VGG16 model= 0.74
Confustion matrix --> 
[[ 6  0  1  1  0  0  1  0  0  0]
 [ 0  8  0  1  1  0  0  0  0  0]
 [ 1  0 10  1  1  0  1  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0]
 [ 0  0  2  0 14  0  1  0  0  0]
 [ 0  0  0  1  0  4  0  0  1  1]
 [ 0  0  2  1  3  0  5  0  0  0]
 [ 0  0  0  0  0  1  0  4  1  1]
 [ 0  0  0  0  1  0  0  0  7  0]
 [ 1  0  0  0  0  0  0  0  0  6]]


Q2 c)

In [128]:
class_mat= multilabel_confusion_matrix(y_true, y_pred)
print("Classwise confusion matrix -> ")
print(class_mat)
for i,arr in enumerate(class_mat):
  acc=np.sum(np.diag(arr)) / np.sum(np.sum(arr,axis=1),axis=0)
  print("classwise accuracy for class ",i," =",acc)

Classwise confusion matrix -> 
[[[89  2]
  [ 3  6]]

 [[90  0]
  [ 2  8]]

 [[81  5]
  [ 4 10]]

 [[85  5]
  [ 0 10]]

 [[77  6]
  [ 3 14]]

 [[92  1]
  [ 3  4]]

 [[86  3]
  [ 6  5]]

 [[93  0]
  [ 3  4]]

 [[90  2]
  [ 1  7]]

 [[91  2]
  [ 1  6]]]
classwise accuracy for class  0  = 0.95
classwise accuracy for class  1  = 0.98
classwise accuracy for class  2  = 0.91
classwise accuracy for class  3  = 0.95
classwise accuracy for class  4  = 0.91
classwise accuracy for class  5  = 0.96
classwise accuracy for class  6  = 0.91
classwise accuracy for class  7  = 0.97
classwise accuracy for class  8  = 0.97
classwise accuracy for class  9  = 0.97


In [124]:
np.sum(np.sum(matrix,axis=1),axis=0)

100

In [74]:
y=next(iter(test_loader))[1]
# z=np.array(y.squeeze(0))
# _,id=torch.max(y.data,1)
_,idx=torch.max(y.data,1)
np.array(idx)



array([8])

In [75]:
len(iter(test_loader)[0])

TypeError: ignored

In [24]:
x[0][0].shape

torch.Size([3, 224, 224])

In [68]:
test_loader = Data.DataLoader(FashionMNIST(test_x,test_y,test_transform), shuffle = True)

In [60]:
y=test_loader[99][1]

In [65]:
y

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])